<a href="https://colab.research.google.com/github/ClarinetInALeatherJacket/OgataLabROP2024/blob/main/WaterfallPlot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import shutil
import codecs
import time
#!pip install numpy
import numpy as np
#!pip install pandas
import pandas as pd
#!pip install matplotlib
import matplotlib.pyplot as plt
from google.colab import files
%load_ext google.colab.data_table
#!pip install ipdb
#import ipdb
#%pdb off

In [ ]:
class FileData:
    zAxisValue: int
    barFrame: pd.DataFrame

    def __init__(self, zAxisValue:int):
        self.zAxisValue = zAxisValue
        self.barFrame = pd.DataFrame

In [ ]:
def parse_input():
    inputs = files.upload()

    output = []

    for key in inputs.keys():
        try: # Try to turn the file name into an integer value by removing the
        # .xlsx extension
            data = FileData(int(key.replace(".xlsx","")))
        except: # If the name isn't formatted properly, throw an error and print
        # the incorrectly formatted name
            print("-"*80)
            print("[ERROR] Bad file name: " + key +
                    ". Please ensure all files are named (numerical value).xlsx.")
            return 0
        data.barFrame = pd.read_excel(f"./{key}")
        data.barFrame.rename(columns={"Bin Center":"Intensity", "# values": "Frequency"}, inplace=True, )
        output.append(data)

    output.sort(key=lambda x: x.zAxisValue, reverse=True)

    return output




In [ ]:
# Source of most of this code: https://matplotlib.org/stable/gallery/mplot3d/bars3d.html#sphx-glr-gallery-mplot3d-bars3d-py
def plot_data(data):

    yticks = np.arange(0, len(data), 1)[::-1] # The -1 here changes the ordering
    # of the Z axis. If it is -1, then the largest value of the Z axis will be
    # at the front of the graph. If the value is 1, then the smallest value of
    # the Z axis will be at the front of the graph
    colours = ["red","forestgreen","darkorange","blue","magenta"]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    colourCount = 0;

    for d, k in zip(data, yticks):

        xs = d.barFrame["Intensity"]
        ys = d.barFrame["Frequency"]
        zs = [k*len(xs)]


        ax.bar(xs, ys, zs=k, zdir="y", color=colours[colourCount], alpha=0.75)
        colourCount = (colourCount + 1) % 5
        print(colourCount)

    ax.set_xlabel("Fluorescent Intensity")
    ax.set_ylabel(input("Please enter the desired Z Axis label:"))
    ax.set_zlabel("Frequency")
    ax.set_ylim(max(yticks),min(yticks))
    ax.set_yticks(yticks)
    ax.set_yticklabels([dataSet.zAxisValue for dataSet in data])


    plt.savefig("./output.png", pad_inches=2.5, dpi=450) #Change DPI value if
    # you want to change the resolution of the image. A higher DPI value means
    # a higher resolution, but also a larger image file. 450 was chosen as a
    # compromise between resolution and size






In [ ]:
if __name__ == "__main__":

    for file in os.listdir("./"):
        if os.path.isfile(f"./{file}"):
            os.remove(f"./{file}")


    data = parse_input()

    if (data != 0):
        plot_data(data)
        files.download("./output.png")
